# Triangular matrices - part 2

Consider the two triangular matrices presented below:

$$\mathbf{U} 
= \left[ \begin{array}{ccccc}
u_{11} & u_{12} & u_{13} & \cdots & u_{1N} \\
       & u_{22} & u_{23} & \cdots & u_{2N} \\
       &        & u_{33} & \cdots & u_{3N}  \\
       &        &        & \ddots & \vdots  \\
       &        &        &        & u_{NN}
\end{array} \right]
$$

and

$$\mathbf{L} 
= \left[ \begin{array}{ccccc}
l_{11} & & & & \\
\vdots & \ddots & & & \\
l_{N-2 \, 1} & \cdots & l_{N-2\,N-2} & &\\
l_{N-1 \, 1} & \cdots & l_{N-1\,N-2} & l_{N-1\,N-1} &\\
l_{N1} & \cdots & l_{N\,N-2} & l_{N\,N-1} & l_{N\,N}
\end{array} \right] \: .
$$

As we can see, a large fraction of the elements forming these matrices is zero. For the case in which $N$ is large, we need to think about **efficient storage schemes**. For example (see the notebook [`diagonal_matrices.ipynb`](https://nbviewer.jupyter.org/github/birocoles/Disciplina-metodos-computacionais/blob/master/Content/diagonal_matrices.ipynb)), a diagonal matrix $\mathbf{D}$ can be efficiently stored as a vector $\mathbf{d}$. Is this particular case, there is an easy relationship between the elements of $\mathbf{d}$ and the elements of the original diagonal matrix $\mathbf{D}$. It is also easy to rewrite the algorithms for computing the product of $\mathbf{D}$ and a full matrix or vector by using the vector $\mathbf{d}$.

In the case of triangular matrices, the storage schemes are more complicated. Let's create the triangular matrices presented above and use them to illustrate different storage schemes. The following cells use the routines [`numpy.random.rand`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html), [`numpy.triu`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.triu.html), [`numpy.tril`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tril.html), and [`numpy.around`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.around.html).

In [1]:
import numpy as np

In [2]:
N = 4
A = np.around(100*np.random.rand(N,N), decimals=3)
U = np.triu(A)
L = np.tril(A)

In [3]:
print A, '\n'
print U, '\n'
print L

[[ 19.712  89.383  57.459  73.792]
 [ 91.905   9.162   8.359  71.278]
 [ 27.108   5.278  22.732   8.701]
 [ 28.94    6.452  25.362  55.512]] 

[[ 19.712  89.383  57.459  73.792]
 [  0.      9.162   8.359  71.278]
 [  0.      0.     22.732   8.701]
 [  0.      0.      0.     55.512]] 

[[ 19.712   0.      0.      0.   ]
 [ 91.905   9.162   0.      0.   ]
 [ 27.108   5.278  22.732   0.   ]
 [ 28.94    6.452  25.362  55.512]]


**1) column-based storage scheme for $\mathbf{U}$**

This scheme stores the triangular matrix $\mathbf{U}$, column by column, in a vector $\mathbf{u}_{c}$ with $N(N+1)/2$ elements. Consider the example below:

$$
\underbrace{\begin{bmatrix}
1 & 2 & 3 \\
0 & 4 & 5 \\
0 & 0 & 6
\end{bmatrix}}_{\mathbf{U}} \rightarrow
\underbrace{\begin{bmatrix}
1 \\ 2 \\ 4 \\ 3 \\ 5 \\ 6
\end{bmatrix}}_{\mathbf{u}_{c}}
$$

An element $ij$ of the matrix $\mathbf{U}$ is associated with the $k$th element of the vector $\mathbf{u}_{c}$, where 

$$
k = i - 1 + j*(j - 1)/2
$$

In [4]:
for j in range(1,N+1):
    for i in range(1,j+1):
        k = i - 1 + j*(j - 1)/2
        print 'ij = %d%d | k = %d' % (i, j, k)

ij = 11 | k = 0
ij = 12 | k = 1
ij = 22 | k = 2
ij = 13 | k = 3
ij = 23 | k = 4
ij = 33 | k = 5
ij = 14 | k = 6
ij = 24 | k = 7
ij = 34 | k = 8
ij = 44 | k = 9


The function below transforms the triangular matrix $\mathbf{U}$ into the vector $\mathbf{u}_{c}$:

In [5]:
def U2uc(U):
    'Transforms the upper triangular matrix U into \
a vector uc by using a column scheme'
    assert U.shape[0] == U.shape[1], 'U must be square'
    # indices of the non-null elements
    i, j = np.triu_indices(U.shape[0])
    # reorganize the elements according to the column scheme
    p = np.argsort(j)
    i = i[p]
    j = j[p]
    # create the vector uc
    uc = U[i, j]
    return uc

In [6]:
uc = U2uc(U)

In [7]:
print uc

[ 19.712  89.383   9.162  57.459   8.359  22.732  73.792  71.278   8.701
  55.512]


In [8]:
print U

[[ 19.712  89.383  57.459  73.792]
 [  0.      9.162   8.359  71.278]
 [  0.      0.     22.732   8.701]
 [  0.      0.      0.     55.512]]


**2) row-based storage scheme for $\mathbf{U}$**

This scheme stores the triangular matrix $\mathbf{U}$, row by row, in a vector $\mathbf{u}_{r}$ with $N(N+1)/2$ elements. Consider the example below:

$$
\underbrace{\begin{bmatrix}
1 & 2 & 3 \\
0 & 4 & 5 \\
0 & 0 & 6
\end{bmatrix}}_{\mathbf{U}} \rightarrow
\underbrace{\begin{bmatrix}
1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
\end{bmatrix}}_{\mathbf{u}_{r}}
$$

An element $ij$ of the matrix $\mathbf{U}$ is associated with the $k$th element of the vector $\mathbf{u}_{r}$, where 

$$
k = j - 1 + (i - 1)*(2*n - i)/2
$$

In [9]:
for i in range(1,N+1):
    for j in range(i,N+1):
        k = j - 1 + (i - 1)*(2*N - i)/2
        print 'ij = %d%d | k = %d' % (i, j, k)

ij = 11 | k = 0
ij = 12 | k = 1
ij = 13 | k = 2
ij = 14 | k = 3
ij = 22 | k = 4
ij = 23 | k = 5
ij = 24 | k = 6
ij = 33 | k = 7
ij = 34 | k = 8
ij = 44 | k = 9


The function below transforms the triangular matrix $\mathbf{U}$ into the vector $\mathbf{u}_{r}$:

In [10]:
def U2ur(U):
    'Transforms the upper triangular matrix U into \
a vector ur by using a row scheme'
    assert U.shape[0] == U.shape[1], 'U must be square'
    # indices of the non-null elements
    i, j = np.triu_indices(U.shape[0])
    # create the vector uc
    ur = U[i, j]
    return ur

In [11]:
ur = U2ur(U)

In [12]:
print ur

[ 19.712  89.383  57.459  73.792   9.162   8.359  71.278  22.732   8.701
  55.512]


In [13]:
print U

[[ 19.712  89.383  57.459  73.792]
 [  0.      9.162   8.359  71.278]
 [  0.      0.     22.732   8.701]
 [  0.      0.      0.     55.512]]


**3) column-based storage scheme for $\mathbf{L}$**

This scheme stores the triangular matrix $\mathbf{L}$, column by column, in a vector $\mathbf{l}_{c}$ with $N(N+1)/2$ elements. Consider the example below:

$$
\underbrace{\begin{bmatrix}
1 & 0 & 0 \\
2 & 3 & 0 \\
4 & 5 & 6
\end{bmatrix}}_{\mathbf{L}} \rightarrow
\underbrace{\begin{bmatrix}
1 \\ 2 \\ 4 \\ 3 \\ 5 \\ 6
\end{bmatrix}}_{\mathbf{l}_{c}}
$$

An element $ij$ of the matrix $\mathbf{L}$ is associated with the $k$th element of the vector $\mathbf{l}_{c}$, where 

$$
k = i - 1 + (j - 1)*(2*n - j)/2
$$

In [14]:
for j in range(1,N+1):
    for i in range(j,N+1):
        k = i - 1 + (j - 1)*(2*N - j)/2
        print 'ij = %d%d | k = %d' % (i, j, k)

ij = 11 | k = 0
ij = 21 | k = 1
ij = 31 | k = 2
ij = 41 | k = 3
ij = 22 | k = 4
ij = 32 | k = 5
ij = 42 | k = 6
ij = 33 | k = 7
ij = 43 | k = 8
ij = 44 | k = 9


The function below transforms the triangular matrix $\mathbf{L}$ into the vector $\mathbf{l}_{c}$:

In [15]:
def L2lc(L):
    'Transforms the lower triangular matrix L into \
a vector lc by using a column scheme'
    assert L.shape[0] == L.shape[1], 'L must be square'
    # indices of the non-null elements
    i, j = np.tril_indices(U.shape[0])
    # reorganize the elements according to the column scheme
    p = np.argsort(j)
    i = i[p]
    j = j[p]
    # create the vector lc
    lc = L[i, j]
    return lc

In [16]:
lc = L2lc(L)

In [17]:
print lc

[ 19.712  91.905  27.108  28.94    9.162   5.278   6.452  22.732  25.362
  55.512]


In [18]:
print L

[[ 19.712   0.      0.      0.   ]
 [ 91.905   9.162   0.      0.   ]
 [ 27.108   5.278  22.732   0.   ]
 [ 28.94    6.452  25.362  55.512]]


**4) row-based storage scheme for $\mathbf{L}$**

This scheme stores the triangular matrix $\mathbf{L}$, row by row, in a vector $\mathbf{l}_{r}$ with $N(N+1)/2$ elements. Consider the example below:

$$
\underbrace{\begin{bmatrix}
1 & 0 & 0 \\
2 & 3 & 0 \\
4 & 5 & 6
\end{bmatrix}}_{\mathbf{L}} \rightarrow
\underbrace{\begin{bmatrix}
1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
\end{bmatrix}}_{\mathbf{l}_{r}}
$$

An element $ij$ of the matrix $\mathbf{L}$ is associated with the $k$th element of the vector $\mathbf{l}_{r}$, where 

$$
k = j - 1 + i*(i - 1)/2
$$

In [19]:
for i in range(1,N+1):
    for j in range(1,i+1):
        k = j - 1 + i*(i - 1)/2
        print 'ij = %d%d | k = %d' % (i, j, k)

ij = 11 | k = 0
ij = 21 | k = 1
ij = 22 | k = 2
ij = 31 | k = 3
ij = 32 | k = 4
ij = 33 | k = 5
ij = 41 | k = 6
ij = 42 | k = 7
ij = 43 | k = 8
ij = 44 | k = 9


The function below transforms the triangular matrix $\mathbf{L}$ into the vector $\mathbf{l}_{r}$:

In [20]:
def L2lr(L):
    'Transforms the lower triangular matrix L into \
a vector lc by using a row scheme'
    assert L.shape[0] == L.shape[1], 'L must be square'
    # indices of the non-null elements
    i, j = np.tril_indices(U.shape[0])
    # create the vector lr
    lr = L[i, j]
    return lr

In [21]:
lr = L2lr(L)

In [22]:
print lr

[ 19.712  91.905   9.162  27.108   5.278  22.732  28.94    6.452  25.362
  55.512]


In [23]:
print L

[[ 19.712   0.      0.      0.   ]
 [ 91.905   9.162   0.      0.   ]
 [ 27.108   5.278  22.732   0.   ]
 [ 28.94    6.452  25.362  55.512]]


These storage schemes can be used to modify the algorithms presented in the previous class ([`triangular_matrices_1.ipynb`](https://nbviewer.jupyter.org/github/birocoles/Disciplina-metodos-computacionais/blob/master/Content/triangular_matrices_1.ipynb)) for computing the products $\mathbf{y} = \mathbf{U} \mathbf{x}$ and $\mathbf{z} = \mathbf{L} \mathbf{x}$. Consider the **Algorithm 3**, for example:

**Algorithm 3**

    for i = 1:N
        y[i] = y[i] + dot(U[i,i:],x[i:])

Notice that **Algorithm 3** accesses the elements of $\mathbf{U}$, row by row. In this case, it is convenient to store $\mathbf{U}$ in a vector $\mathbf{u}_{r}$, according to the second storage scheme presented above. Finally, the **Algorithm 3** can be modified as follows:

**Algorithm 3 (modified)**

    for i = 1:N
        k1 = i - 1 + (i - 1)*(2*N - i)/2
        k2 = N - 1 + (i - 1)*(2*N - i)/2
        y[i] = y[i] + dot(ur[k1:k2],x[i:])

### Exercise

1. In your `my_functions.py` file, implement the modified versions of algorithms 3, 5, 8, and 10, according to the storage schemes presented here. Each modified algorithm must be implemented in a single function. Each modified algorithm must receive the proper vector (uc, ur, lc, or lr) and a vector x and return the resultant vector y or z.
2. In your `test_my_functions.py` file, create four tests to compare the results produced by the algorithms 3, 5, 8, and 10 (presented in the notebook [`triangular_matrices_1.ipynb`](https://nbviewer.jupyter.org/github/birocoles/Disciplina-metodos-computacionais/blob/master/Content/triangular_matrices_1.ipynb)) and their modified versions created here, in the item 1.

### References

* Golub, G. H. and Van Loan, C. F. Matrix computations, 4th edition, Johns Hopkins University Press, 2013

* [IBM Knowledge Center - Engineering and Scientific Subroutine Library](https://www.ibm.com/support/knowledgecenter/en/SSFHY8_5.5.0/com.ibm.cluster.essl.v5r5.essl100.doc/am5gr_data.htm)

* [Intel developer zone - Matrix Storage Schemes for LAPACK Routines](https://software.intel.com/en-us/mkl-developer-reference-c-matrix-storage-schemes-for-lapack-routines)